<a href="https://colab.research.google.com/github/wonkwonlee/a2w-kma-big-data-contest/blob/main/XGBoost_undersample.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#pip install xgboost

In [41]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [42]:
# # 경로 설정 
# import os
# os.chdir('C:\\Users\\billk\\weathercontest\\data')
import os
os.chdir('/content/drive/MyDrive/Colab Notebooks')

In [ ]:
import tensorflow as tf
from tensorflow import keras

import os
import tempfile

import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

import sklearn
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import make_scorer, accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from sklearn.linear_model import LogisticRegression


from xgboost import plot_importance
from xgboost import XGBClassifier

In [ ]:
def metrics(y_test, pred):
    accuracy = accuracy_score(y_test, pred)
    precision = precision_score(y_test, pred)
    recall = recall_score(y_test, pred)
    f1 = f1_score(y_test, pred)
    roc_score = roc_auc_score(y_test, pred)
    print('정확도 : {0:.2f}, 정밀도 : {1:.2f}, 재현율 : {2:.2f}'.format(accuracy, precision, recall))
    print('f1-score : {0:.2f}, auc : {1:.2f}'.format(f1, roc_score))

# 데이터 처리 및 탐색

In [ ]:
raw_df = pd.read_csv("../A2W/데이터/data_submission/merge_final_real.csv", encoding='utf-8-sig')

In [ ]:
raw_df['1day_yn'] = 0.0
raw_df['2day_yn'] = 0.0

In [ ]:
# 열을 밀어서 출력, 맨 첫날, 마지막 날의 경우 산사태가 발생하지 않음.
raw_df['1day_yn'][1:-1] = raw_df['landslide'][0:-2]
raw_df['2day_yn'][2:-1] = raw_df['landslide'][0:-3]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [ ]:
select_df = raw_df[['maxTa','minTa','avgTa','EIGHT_AGL','stnId','mi10MaxRn', 'hr1MaxRn', 'sumRnDur', 'sumRn', 'maxInsWs', 'maxWs', 'avgWs','minRhm','avgRhm','n99Rn', 'PRRCK_LARG', 'PRRCK_MDDL','LOCTN_ALTT','LOCTN_GRDN','CLZN_CD','TPGRP_TPCD','PRDN_FOM_C','SLDPT_TPCD','SLANT_TYP','SCSTX_CD','SLTP_CD','STORUNST','FROR_CD','FRTP_CD','KOFTR_GROU','DMCLS_CD','AGCLS_CD','HEIGHT','1day_yn']]
cleaned_df_1 = select_df.fillna(0)
cleaned_df_1.info()
cleaned_df_1.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2093790 entries, 0 to 2093789
Data columns (total 34 columns):
 #   Column      Dtype  
---  ------      -----  
 0   maxTa       float64
 1   minTa       float64
 2   avgTa       float64
 3   EIGHT_AGL   float64
 4   stnId       int64  
 5   mi10MaxRn   float64
 6   hr1MaxRn    float64
 7   sumRnDur    float64
 8   sumRn       float64
 9   maxInsWs    float64
 10  maxWs       float64
 11  avgWs       float64
 12  minRhm      float64
 13  avgRhm      float64
 14  n99Rn       float64
 15  PRRCK_LARG  int64  
 16  PRRCK_MDDL  int64  
 17  LOCTN_ALTT  float64
 18  LOCTN_GRDN  float64
 19  CLZN_CD     int64  
 20  TPGRP_TPCD  int64  
 21  PRDN_FOM_C  int64  
 22  SLDPT_TPCD  int64  
 23  SLANT_TYP   int64  
 24  SCSTX_CD    int64  
 25  SLTP_CD     int64  
 26  STORUNST    float64
 27  FROR_CD     float64
 28  FRTP_CD     float64
 29  KOFTR_GROU  float64
 30  DMCLS_CD    float64
 31  AGCLS_CD    float64
 32  HEIGHT      float64
 33  1day_yn

,maxTa,minTa,avgTa,EIGHT_AGL,stnId,mi10MaxRn,hr1MaxRn,sumRnDur,sumRn,maxInsWs,maxWs,avgWs,minRhm,avgRhm,n99Rn,PRRCK_LARG,PRRCK_MDDL,LOCTN_ALTT,LOCTN_GRDN,CLZN_CD,TPGRP_TPCD,PRDN_FOM_C,SLDPT_TPCD,SLANT_TYP,SCSTX_CD,SLTP_CD,STORUNST,FROR_CD,FRTP_CD,KOFTR_GROU,DMCLS_CD,AGCLS_CD,HEIGHT,1day_yn
0,2.1,-1.1,0.8,0.0,115,0.0,0.0,19.33,28.0,29.4,19.0,11.1,65.0,82.5,5.8,0,0,0.0,0.0,0,0,0,0,0,0,0,1.0,2.0,2.0,30.0,1.0,5.0,16.0,0.0
1,1.3,-0.9,0.1,0.0,115,0.0,0.0,22.83,24.8,14.5,8.5,6.0,65.0,91.5,29.7,0,0,0.0,0.0,0,0,0,0,0,0,0,1.0,2.0,2.0,30.0,1.0,5.0,16.0,0.0
2,0.8,-1.4,-0.3,0.0,115,0.0,0.0,12.83,7.5,15.9,10.2,6.5,61.0,80.3,0.3,0,0,0.0,0.0,0,0,0,0,0,0,0,1.0,2.0,2.0,30.0,1.0,5.0,16.0,0.0
3,3.9,0.4,1.7,0.0,115,0.0,0.0,0.00,0.0,11.2,6.8,3.1,53.0,63.1,0.0,0,0,0.0,0.0,0,0,0,0,0,0,0,1.0,2.0,2.0,30.0,1.0,5.0,16.0,0.0
4,2.6,-1.2,1.2,0.0,115,0.0,0.0,7.25,1.8,13.9,6.9,4.1,42.0,61.5,12.3,0,0,0.0,0.0,0,0,0,0,0,0,0,1.0,2.0,2.0,30.0,1.0,5.0,16.0,0.0


## 1일 후 예측

In [ ]:
y = cleaned_df_1['1day_yn']
X = cleaned_df_1.drop('1day_yn', axis =1)
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 11)
# print(X_train.shape, X_test.shape)

In [ ]:
from imblearn.under_sampling import RandomUnderSampler

# 1: 50 비율
X_samp, y_samp = RandomUnderSampler(random_state=1024, ratio={1: 359, 0: 17950}).fit_sample(X, y)
X_df = pd.DataFrame(X_samp)
y_df = pd.DataFrame(y_samp)

df_samp = pd.concat([X_df, y_df], axis=1)

/usr/local/lib/python3.7/dist-packages/sklearn/externals/six.py:31: FutureWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.neighbors.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.neighbors. Anything that cannot be imported from sklearn.neighbors is now part of the private API.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)


In [ ]:
y_df.value_counts()

0.0    17950
1.0      359
dtype: int64

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_df, y_df, test_size=0.25, shuffle=True, stratify=y_df, random_state=1024)

print('X_train shape:', X_train.shape)
print('X_test shape:', X_test.shape)
print('y_train shape:', y_train.shape)
print('y_test shape:', y_test.shape)

X_train shape: (13731, 33)
X_test shape: (4578, 33)
y_train shape: (13731, 1)
y_test shape: (4578, 1)


In [ ]:
xgb = XGBClassifier(n_estimators=500, learning_rate = 0.1, max_depth = 4)
xgb.fit(X_train, y_train)
xgb_pred = xgb.predict(X_test)

/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:235: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:268: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [ ]:
metrics(y_test, xgb_pred)

정확도 : 1.00, 정밀도 : 0.94, 재현율 : 0.84
f1-score : 0.89, auc : 0.92


### 하이퍼 파라미터 튜닝 후 예측

In [ ]:
xgb = XGBClassifier()

xgb_param_grid = {
    'n_estimators' : [100, 200, 400, 600],
    'learning_rate' : [0.01, 0.05, 0.1, 0.15, 0.2],
    'max_depth' : [4, 6, 8, 10, 12],
}
# sklearn.metrics.SCORERS.keys()
xgb_grid = GridSearchCV(xgb, param_grid = xgb_param_grid, scoring="roc_auc", n_jobs= 1, verbose = 1)
xgb_grid.fit(X_train, y_train)

In [ ]:
print("최고 평균 auc : {0:.4f}".format(xgb_grid.best_score_))
print("최고의 파라미터 : ", xgb_grid.best_params_)
print("최고 점수를 낸 파라미터를 가진 모형 : ", xgb_grid.best_estimator_)

최고 평균 auc : 0.9951
최고의 파라미터 :  {'learning_rate': 0.05, 'max_depth': 10, 'n_estimators': 200}


In [ ]:
xgb_grid.best_score_

0.9951095410085461

In [ ]:
xgb_grid.best_params_

{'learning_rate': 0.05, 'max_depth': 10, 'n_estimators': 200}

In [ ]:
result = pd.DataFrame(xgb_grid.cv_results_)
result.sort_values(by=['rank_test_score'], inplace=True)

In [ ]:
result.head(10)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_learning_rate,param_max_depth,param_n_estimators,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
33,4.357487,0.025953,0.019488,0.001476,0.05,10,200,"{'learning_rate': 0.05, 'max_depth': 10, 'n_es...",0.994072,0.998522,0.997503,0.996767,0.988684,0.995110,0.003535,1
34,7.948726,0.108633,0.042448,0.001466,0.05,10,400,"{'learning_rate': 0.05, 'max_depth': 10, 'n_es...",0.993123,0.998627,0.997400,0.997338,0.988072,0.994912,0.003895,2
37,4.674980,0.032013,0.019109,0.000496,0.05,12,200,"{'learning_rate': 0.05, 'max_depth': 12, 'n_es...",0.994231,0.997905,0.997070,0.997310,0.987831,0.994869,0.003741,3
29,3.861755,0.049862,0.018542,0.000491,0.05,8,200,"{'learning_rate': 0.05, 'max_depth': 8, 'n_est...",0.993839,0.998536,0.997166,0.997014,0.987700,0.994851,0.003893,4
19,14.430042,0.097200,0.039429,0.001639,0.01,12,600,"{'learning_rate': 0.01, 'max_depth': 12, 'n_es...",0.993605,0.997289,0.997248,0.997771,0.988175,0.994818,0.003642,5
52,2.185093,0.023568,0.012373,0.001278,0.1,10,100,"{'learning_rate': 0.1, 'max_depth': 10, 'n_est...",0.993247,0.997800,0.997255,0.997420,0.988312,0.994807,0.003644,6
48,1.954566,0.035377,0.011836,0.000485,0.1,8,100,"{'learning_rate': 0.1, 'max_depth': 8, 'n_esti...",0.994698,0.998333,0.996939,0.997186,0.986737,0.994779,0.004190,7
15,12.864836,0.178590,0.036181,0.001325,0.01,10,600,"{'learning_rate': 0.01, 'max_depth': 10, 'n_es...",0.993715,0.997506,0.997317,0.996615,0.988450,0.994721,0.003418,8
49,3.760266,0.022624,0.025723,0.000987,0.1,8,200,"{'learning_rate': 0.1, 'max_depth': 8, 'n_esti...",0.993715,0.998431,0.997331,0.997606,0.986249,0.994666,0.004509,9
56,2.337781,0.014514,0.011894,0.000185,0.1,12,100,"{'learning_rate': 0.1, 'max_depth': 12, 'n_est...",0.993763,0.998052,0.997503,0.997056,0.986895,0.994654,0.004157,10


In [ ]:
result[['params', 'mean_test_score', 'rank_test_score']].head(10)

,params,mean_test_score,rank_test_score
33,"{'learning_rate': 0.05, 'max_depth': 10, 'n_es...",0.995110,1
34,"{'learning_rate': 0.05, 'max_depth': 10, 'n_es...",0.994912,2
37,"{'learning_rate': 0.05, 'max_depth': 12, 'n_es...",0.994869,3
29,"{'learning_rate': 0.05, 'max_depth': 8, 'n_est...",0.994851,4
19,"{'learning_rate': 0.01, 'max_depth': 12, 'n_es...",0.994818,5
52,"{'learning_rate': 0.1, 'max_depth': 10, 'n_est...",0.994807,6
48,"{'learning_rate': 0.1, 'max_depth': 8, 'n_esti...",0.994779,7
15,"{'learning_rate': 0.01, 'max_depth': 10, 'n_es...",0.994721,8
49,"{'learning_rate': 0.1, 'max_depth': 8, 'n_esti...",0.994666,9
56,"{'learning_rate': 0.1, 'max_depth': 12, 'n_est...",0.994654,10


In [ ]:
import joblib

xgb_best = xgb_grid.best_estimator_
xgb_best.fit(X_train, y_train)
joblib.dump(xgb_best, './xgb_best_model_1day_yn.pkl')

/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:235: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:268: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


['./xgb_best_model_1day_yn.pkl']

In [ ]:
xgb_best_pred = xgb_best.predict(X_test)
metrics(y_test, xgb_best_pred)

정확도 : 1.00, 정밀도 : 0.96, 재현율 : 0.81
f1-score : 0.88, auc : 0.91


## 최고의 파라미터 값으로 예측 실시 
n_estimators = 200, learning_rate = 0.05, max_depth = 10

In [ ]:
xgb = XGBClassifier(n_estimators=100, learning_rate = 0.2, max_depth = 6)
evals = [(X_test, y_test)]
xgb.fit(X_train, y_train, early_stopping_rounds = 100, eval_metric = "logloss", eval_set = evals, verbose = 1)

## 2일 후 예측

In [ ]:
#분석을 위해 음수 값을 가지는 칼럼들을 제외하고 결측치를 0으로 대체한다.
select_df = raw_df[['maxTa','minTa','avgTa','EIGHT_AGL','stnId','mi10MaxRn', 'hr1MaxRn', 'sumRnDur', 'sumRn', 'maxInsWs', 'maxWs', 'avgWs','minRhm','avgRhm','n99Rn', 'PRRCK_LARG', 'PRRCK_MDDL','LOCTN_ALTT','LOCTN_GRDN','CLZN_CD','TPGRP_TPCD','PRDN_FOM_C','SLDPT_TPCD','SLANT_TYP','SCSTX_CD','SLTP_CD','STORUNST','FROR_CD','FRTP_CD','KOFTR_GROU','DMCLS_CD','AGCLS_CD','HEIGHT','2day_yn']]
cleaned_df_2 = select_df.fillna(0)

In [ ]:
y = cleaned_df_2['2day_yn']
X = cleaned_df_2.drop('2day_yn', axis =1)

In [ ]:
# 1: 50 비율
X_samp, y_samp = RandomUnderSampler(random_state=1024, ratio={1: 359, 0: 17950}).fit_sample(X, y)
X_df = pd.DataFrame(X_samp)
y_df = pd.DataFrame(y_samp)

df_samp = pd.concat([X_df, y_df], axis=1)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_df, y_df, test_size=0.25, shuffle=True, stratify=y_df, random_state=1024)

print('X_train shape:', X_train.shape)
print('X_test shape:', X_test.shape)
print('y_train shape:', y_train.shape)
print('y_test shape:', y_test.shape)

X_train shape: (13731, 33)
X_test shape: (4578, 33)
y_train shape: (13731, 1)
y_test shape: (4578, 1)


In [ ]:
xgb = XGBClassifier()

xgb_param_grid = {
    'n_estimators' : [100, 200, 400, 600],
    'learning_rate' : [0.01, 0.05, 0.1, 0.15, 0.2],
    'max_depth' : [4, 6, 8, 10, 12],
}
# sklearn.metrics.SCORERS.keys()
xgb_grid = GridSearchCV(xgb, param_grid = xgb_param_grid, scoring="roc_auc", n_jobs= 1, verbose = 1)
xgb_grid.fit(X_train, y_train)

Fitting 5 folds for each of 100 candidates, totalling 500 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:235: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:268: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:235: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:268: DataConversionWarning: A column-vector y was pas

In [43]:
print("최고 평균 auc : {0:.4f}".format(xgb_grid.best_score_))
print("최고의 파라미터 : ", xgb_grid.best_params_)
print("최고 점수를 낸 파라미터를 가진 모형 : ", xgb_grid.best_estimator_)

최고 평균 auc : 0.9915
최고의 파라미터 :  {'learning_rate': 0.05, 'max_depth': 10, 'n_estimators': 400}
최고 점수를 낸 파라미터를 가진 모형 :  XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.05, max_delta_step=0, max_depth=10,
              min_child_weight=1, missing=None, n_estimators=400, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)


In [44]:
result = pd.DataFrame(xgb_grid.cv_results_)
result.sort_values(by=['rank_test_score'], inplace=True)

In [45]:
result[['params', 'mean_test_score', 'rank_test_score']].head(10)

,params,mean_test_score,rank_test_score
34,"{'learning_rate': 0.05, 'max_depth': 10, 'n_es...",0.991461,1
96,"{'learning_rate': 0.2, 'max_depth': 12, 'n_est...",0.991420,2
72,"{'learning_rate': 0.15, 'max_depth': 10, 'n_es...",0.991408,3
57,"{'learning_rate': 0.1, 'max_depth': 12, 'n_est...",0.991347,4
52,"{'learning_rate': 0.1, 'max_depth': 10, 'n_est...",0.991309,5
53,"{'learning_rate': 0.1, 'max_depth': 10, 'n_est...",0.991274,6
33,"{'learning_rate': 0.05, 'max_depth': 10, 'n_es...",0.991268,7
88,"{'learning_rate': 0.2, 'max_depth': 8, 'n_esti...",0.991252,8
92,"{'learning_rate': 0.2, 'max_depth': 10, 'n_est...",0.991214,9
35,"{'learning_rate': 0.05, 'max_depth': 10, 'n_es...",0.991205,10


In [46]:
import joblib

xgb_best = xgb_grid.best_estimator_
xgb_best.fit(X_train, y_train)
joblib.dump(xgb_best, './xgb_best_model_2day_yn.pkl')

/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:235: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:268: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


['./xgb_best_model_2day_yn.pkl']

In [47]:
xgb_best_pred = xgb_best.predict(X_test)
metrics(y_test, xgb_best_pred)

정확도 : 1.00, 정밀도 : 0.97, 재현율 : 0.79
f1-score : 0.87, auc : 0.89


# XGBoost 결과
## 1일 뒤 하이퍼 파라미터
* n_estimators = 200
* learning_rate = 0.05
* max_depth = 10

## 1일 뒤 예측
* 정확도: 1.00
* 정밀도: 0.96
* 재현율: 0.81
* f1-score: 0.88
* AUC: 0.91

## 2일 뒤 하이퍼 파라미터
* n_estimators = 400
* learning_rate = 0.05
* max_depth = 10

## 2일 뒤 예측
* 정확도: 1.00
* 정밀도: 0.97
* 재현율: 0.79
* f1-score: 0.87
* AUC: 0.89